In [1]:
from qiskit import QuantumCircuit, Aer, execute, IBMQ, QuantumRegister
from qiskit.visualization import plot_histogram, plot_state_qsphere
import math
import numpy as np

# Evaluation of the effectiveness of error mitigation
This notebook is made to help us compare the result noise between a noisy measurement and a noisy measurement with error mitigation.
The way we compare these two vectors is the following:
* We set up the a noise model (here the noise model is a Pauli error, that introduces an X gate after measurement, with probability p).
* We compute our error mitigation matrix using Qiskit's Ignis module (soon to be deprecated, see Qiskit experiments).
* We introduce a simple circuit to make the measurements on. Here, the example is using the GHZ on two qubits.
* We repeat the experience of measuring the bitstring resulting from the circuit and correct it.
* For each iterations, we compute the average of the errors relative to the known truth for the noisy and mitigated measurement.
* After the iterations, we average these errors to have the average error correction.

**Note :** 
* As seen in the cell below, the backend used is the qasm_simulator. One can use whichever simulator or real backends he wants to test on.
* The noise model generates an X gate on the measurement with a probability of $\frac{1}{10}$. One can tune this parameter in the argument of the get_noise function, or even give a totaly different noise model to the execute functions.
* One can of course tune the circuit he wants the error mitigation to be tested on. If done so, the truth dict needs to be changed as well to match the expected output of the new circuit.

In [2]:
def get_noise(p):
    """
    Generates a NoiseModel object which applies an X gate to a qubit with probability p
    Input : p (float) the probability of applying an X gate
    Output : noise_model (qiskit.providers.aer.noise.NoiseModel) the noise model object that can be given to our simulator
    """
    from qiskit.providers.aer.noise import NoiseModel
    from qiskit.providers.aer.noise.errors import pauli_error
    error_meas = pauli_error([('X',p), ('I', 1 - p)]) #The pauli error is a class that applies the gates we gave it with the chances associated

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_meas, "measure") # measurement error is applied to measurements

    return noise_model

noise_model = get_noise(0.1)#We introduce an X gate with probability 0.1

In [12]:
#In the variable backend, you can define whichever simulator or real backend you want to use
backend = Aer.get_backend('qasm_simulator')

The cells below define the error mitigation matrix thanks to qiskit's ignis library.

In [4]:
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
qr = QuantumRegister(2)#We create 2 quantum registers as our example is in C^4 
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')

C:\Users\108997~1\AppData\Local\Temp/ipykernel_9476/3289176334.py:1: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter


In [5]:
job = execute(meas_calibs, backend, noise_model = noise_model)
result = job.result()#We are executing our circuits to have the distribution of the errors

meas_fitter = CompleteMeasFitter(result, state_labels, circlabel='mcal')#The results are fitted into a matrix
np.around(meas_fitter.cal_matrix,3)#The error mitigation matrix is displayed, with its coefficients rounded to make it more readable

array([[0.825, 0.095, 0.079, 0.006],
       [0.075, 0.803, 0.011, 0.084],
       [0.097, 0.009, 0.825, 0.096],
       [0.003, 0.094, 0.085, 0.814]])

In [6]:
#Define the quantum circuit you want to test error mitigation on here : 
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)
qc.measure_all()

In [7]:
# Get the filter object
meas_filter = meas_fitter.filter

In [8]:
def fill_dict(d):
    """
    This procedure allows an user to fill a dictionnary with the missing basis vector
    Input: a dictionnary d with incomplete basis
    Modified input : a dictionnary with the whole basis in it. For example, if we input d = {'00' : 994, '01' : 3, '11': 3},
    the state of d at the end of the procedure will be d = {'00' : 994, '01' : 3, '10' : 0, '11': 3}
    """
    item_amount = len(d)
    bit_size = len(list(d.keys())[0])
    if item_amount != 2**bit_size:
        bases_list = [format(i,"b").zfill(bit_size) for i in range(2**bit_size)]
        key_list = list(d.keys())
        for bases in bases_list:
            if bases not in key_list:
                d[bases] = 0
    

In [9]:
#One can set the amount of iteration to evaluate the precision gain of the error mitigation algorithm
ite = 100
noisy_average = []
mitigated_average = []
#This variable must be set according to the circuit you are evaluating. Here, as we aevaluate a ghz, we have the truth :
truth = {'00' : 500, '01':0, '10':0, '11':500}

for _ in range(ite):
    job = execute(qc,backend,noise_model = noise_model, shots = 1024)
    result = job.result()
    noisy_counts = result.get_counts()
    mitigated_results = meas_filter.apply(result)
    mitigated_counts = mitigated_results.get_counts()
    fill_dict(noisy_counts)#We fill the missing values from the results
    fill_dict(mitigated_counts)
    
    item_amount = len(noisy_counts)
    bit_size = len(list(noisy_counts.keys())[0])
    bases_list = bases_list = [format(i,"b").zfill(bit_size) for i in range(2**bit_size)]
    dist_noisy = {}
    dist_mitigated = {}
    for bases in bases_list :
        dist_noisy[bases] = abs(truth[bases] - noisy_counts[bases])#In each dict, we store the distance of the bitstring measurement to the true measurement.
        dist_mitigated[bases] = abs(truth[bases] - mitigated_counts[bases])#To compare the noisy and mitigated result, we store the result in two separate dicts
    noisy_average.append(np.average(list(dist_noisy.values())))#We average the errors over the different basis bitstring.
    mitigated_average.append(np.average(list(dist_mitigated.values())))
    
print("Average errors on noisy measurement : ",np.average(noisy_average)/1024)#We average the results of each experiments and divide it by our number of shots to get the average distance to the truth
print("Average errors on mitigated measurement : ",np.average(mitigated_average)/1024)
    
    
        
        
        
    
    
    

Average errors on noisy measurement :  0.0841259765625
Average errors on mitigated measurement :  0.011405381797309893


The results above shows the averaged error correction with and without error mitigation. We can observe that even if there is quite a progress (on average the results are seven times less noisy), there are still some errors that persist. These results can be associated with the amount of shots for each <em>execute</em> call.